# Eliminar valores perdidos o anómalos

En muchas ocasiones hay datos que no se han podido obtener. Ya sea porque el proceso de medida haya fallado, porque se han perdido en la comunicación o por cualquier otra razón, muchas veces nos encontramos con datos que faltan marcados como nulos. En otras ocasiones somos nosotros los que al detectar un valor anómalo lo marcamos como nulo.

Si nuestros datos contienen valores marcados como nulos (ya sea con `np.nan`, `None`, un `-1` en características que son sólo positivas o cualquier otra marca) y el algoritmo de aprendizaje no puede trabajar con ellos, podemos seguir varias estrategias:

* Eliminar los objetos (filas) con valores nulos
* Eliminar las caracteristicas (columnas) con valores nulos
* Sustituir los valores nulos por valores que estimemos adecuados

En este tema vamos a ver las dos primeras y la tercera se verá en el tema 2.5.

## Valores atípicos como valores nulos

Como ejemplo, vamos a suponer que queremos descartar los valores anómalos detectados en el mismo conjunto de ejemplo del tema 1.6.

In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets
dataset = datasets.fetch_openml(name='plasma_retinol', version=2, as_frame=True)
tabla = dataset.frame
tabla

,AGE,SEX,SMOKSTAT,QUETELET,VITUSE,CALORIES,FAT,FIBER,ALCOHOL,CHOLESTEROL,BETADIET,RETDIET,BETAPLASMA,binaryClass
0,64.0,Female,Former,21.48380,Yes_fairly_often,1298.8,57.0,6.3,0.0,170.3,1945.0,890.0,200.0,N
1,76.0,Female,Never,23.87631,Yes_fairly_often,1032.5,50.1,15.8,0.0,75.8,2653.0,451.0,124.0,N
2,38.0,Female,Former,20.01080,Yes_not_often,2372.3,83.6,19.1,14.1,257.9,6321.0,660.0,328.0,N
3,40.0,Female,Former,25.14062,No,2449.5,97.5,26.5,0.5,332.6,1061.0,864.0,153.0,N
4,72.0,Female,Never,20.98504,Yes_fairly_often,1952.1,82.6,16.2,0.0,170.8,2863.0,1209.0,92.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,46.0,Female,Former,25.89669,No,2263.6,98.2,19.4,2.6,306.5,2572.0,1261.0,164.0,P
311,45.0,Female,Never,23.82703,Yes_fairly_often,1841.1,84.2,14.1,2.2,257.7,1665.0,465.0,80.0,P
312,49.0,Female,Never,24.26126,Yes_fairly_often,1125.6,44.8,11.9,4.0,150.5,6943.0,520.0,300.0,P
313,31.0,Female,Former,23.45255,Yes_fairly_often,2729.6,144.4,13.2,2.2,381.8,741.0,644.0,121.0,N


En el siguiente código hacemos lo mismo que en el ejemplo del tema 1.6 pero, en vez de imprimir los valores que se salen del rango, los marcamos cambiandolos por [`np.nan`](https://numpy.org/doc/stable/reference/constants.html#numpy.nan) (*Not a Number*) que es el valor de Numpy para los valores nulos. Nota: se puede ver escrito como `np.NAN` o `np.NaN` en algunos códigos. Son equivalentes pero [la forma recomendada](https://numpy.org/doc/stable/reference/constants.html#numpy.NaN) es `np.nan`.

Además, vamos a anotar las columnas en las que aparecen los valores atípicos que hemos marcado como nulos (las filas ya las anotabamos). Como las columnas se van a visitar varias veces para evitar duplicados de forma eficiente usaremos la estructura de datos [`set`](https://docs.python.org/3/library/stdtypes.html#set) de Python.

In [2]:
umbral_z_score = 4.0

# Calcular z-score, marcando atípicos con np.nan
filas_atipicos = []
columnas_atipicos = set()
desc = tabla.describe()
for i, fila in tabla.iterrows():
    atipico_detectado = False
    for caract in desc:
        z_score = abs(tabla.loc[i][caract] - desc.loc['mean'][caract]) / desc.loc['std'][caract]
        if z_score > umbral_z_score:
            atipico_detectado = True
            columnas_atipicos.add(caract)
            tabla.loc[i,caract] = np.nan
    if atipico_detectado:
        filas_atipicos.append(i)
        
filas_atipicos

[39, 50, 61, 93, 151, 170, 207, 218, 225, 256, 261, 262, 308]

In [3]:
columnas_atipicos

{'ALCOHOL',
 'BETADIET',
 'BETAPLASMA',
 'CALORIES',
 'CHOLESTEROL',
 'FAT',
 'FIBER',
 'QUETELET',
 'RETDIET'}

Podemos ver por ejemplo los cuatro valores nulos de las filas 256, 261 y 262 que Pandas muestra como 'NaN':

In [4]:
tabla.iloc[256:263]

,AGE,SEX,SMOKSTAT,QUETELET,VITUSE,CALORIES,FAT,FIBER,ALCOHOL,CHOLESTEROL,BETADIET,RETDIET,BETAPLASMA,binaryClass
256,40.0,Female,Never,31.24219,Yes_fairly_often,3014.9,165.7,14.4,0.0,NaN,1028.0,3061.0,0.0,P
257,29.0,Female,Never,37.93996,Yes_fairly_often,1631.0,55.6,13.8,0.5,189.5,3435.0,1104.0,84.0,N
258,71.0,Female,Former,24.98825,No,1399.5,66.5,9.6,8.0,260.0,1527.0,822.0,161.0,N
259,45.0,Female,Never,23.43164,Yes_fairly_often,2319.0,122.1,13.4,0.1,305.7,2047.0,1125.0,331.0,N
260,63.0,Female,Never,18.92094,No,1655.9,70.8,15.1,0.1,177.3,2897.0,505.0,366.0,P
261,46.0,Female,Former,24.26126,Yes_not_often,1422.8,58.3,7.8,7.1,206.3,1987.0,608.0,NaN,P
262,75.0,Female,Never,21.67837,Yes_fairly_often,2511.5,92.3,NaN,0.6,228.3,4271.0,916.0,NaN,P


Como en el código hemos extraído las filas en las que encontramos valores atípicos, podemos eliminarla directamente usando el [método drop](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html):

In [5]:
tabla_sin_filas_atipicas = tabla.drop(filas_atipicos, axis=0)
tabla_sin_filas_atipicas

,AGE,SEX,SMOKSTAT,QUETELET,VITUSE,CALORIES,FAT,FIBER,ALCOHOL,CHOLESTEROL,BETADIET,RETDIET,BETAPLASMA,binaryClass
0,64.0,Female,Former,21.48380,Yes_fairly_often,1298.8,57.0,6.3,0.0,170.3,1945.0,890.0,200.0,N
1,76.0,Female,Never,23.87631,Yes_fairly_often,1032.5,50.1,15.8,0.0,75.8,2653.0,451.0,124.0,N
2,38.0,Female,Former,20.01080,Yes_not_often,2372.3,83.6,19.1,14.1,257.9,6321.0,660.0,328.0,N
3,40.0,Female,Former,25.14062,No,2449.5,97.5,26.5,0.5,332.6,1061.0,864.0,153.0,N
4,72.0,Female,Never,20.98504,Yes_fairly_often,1952.1,82.6,16.2,0.0,170.8,2863.0,1209.0,92.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,46.0,Female,Former,25.89669,No,2263.6,98.2,19.4,2.6,306.5,2572.0,1261.0,164.0,P
311,45.0,Female,Never,23.82703,Yes_fairly_often,1841.1,84.2,14.1,2.2,257.7,1665.0,465.0,80.0,P
312,49.0,Female,Never,24.26126,Yes_fairly_often,1125.6,44.8,11.9,4.0,150.5,6943.0,520.0,300.0,P
313,31.0,Female,Former,23.45255,Yes_fairly_often,2729.6,144.4,13.2,2.2,381.8,741.0,644.0,121.0,N


Y de la misma forma, podríamos eliminar las columnas.

In [6]:
tabla_sin_columnas_atipicas = tabla.drop(columnas_atipicos, axis=1)
tabla_sin_columnas_atipicas

,AGE,SEX,SMOKSTAT,VITUSE,binaryClass
0,64.0,Female,Former,Yes_fairly_often,N
1,76.0,Female,Never,Yes_fairly_often,N
2,38.0,Female,Former,Yes_not_often,N
3,40.0,Female,Former,No,N
4,72.0,Female,Never,Yes_fairly_often,N
...,...,...,...,...,...
310,46.0,Female,Former,No,P
311,45.0,Female,Never,Yes_fairly_often,P
312,49.0,Female,Never,Yes_fairly_often,P
313,31.0,Female,Former,Yes_fairly_often,N


Como veis, eliminar las columnas suele ser mucho más drástico que eliminar las filas con valores nulos. En este caso perdemos la mitad de la información. Mientras que, eliminando las filas sólo se perdía un 4% ($\frac{13}{315}$) de los datos. No obstante, puede haber casos donde pocas columnas sean las problemáticas, por ejemplo, porque sean características más difíciles de medir. En esos casos puede que no sean características interesantes para el procesado posterior y merezca la pena probar sin ellas.

## Conjuntos de datos con valores perdidos

Nos podemos encontrar con conjuntos de dato que tienen algunos valores marcados como nulos, de la misma forma que hicimos nosotros en el caso anterior pero, en ese caso, no tenemos la lista de filas o columnas con valores nulos.

In [7]:
dataset = datasets.fetch_openml(name='sleep', version=2, as_frame=True)
tabla_sleep = dataset.frame
tabla_sleep

,body_weight,brain_weight,slow_wave,paradoxical,total_sleep,maximum_life_span,gestation_time,predation_index,sleep_exposure_index,overall_danger_index
0,6654.000,5712.0,NaN,NaN,3.3,38.6,645.0,3.0,5.0,3.0
1,1.000,6.6,6.3,2.0,8.3,4.5,42.0,3.0,1.0,3.0
2,3.385,44.5,NaN,NaN,12.5,14.0,60.0,1.0,1.0,1.0
3,0.920,5.7,NaN,NaN,16.5,NaN,25.0,5.0,2.0,3.0
4,2547.000,4603.0,2.1,1.8,3.9,69.0,624.0,3.0,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...
57,2.000,12.3,4.9,0.5,5.4,7.5,200.0,3.0,1.0,3.0
58,0.104,2.5,13.2,2.6,15.8,2.3,46.0,3.0,2.0,2.0
59,4.190,58.0,9.7,0.6,10.3,24.0,210.0,4.0,3.0,4.0
60,3.500,3.9,12.8,6.6,19.4,3.0,14.0,2.0,1.0,1.0


**Ejercicio**: Usando la función [`pd.isnull()` de Pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.isnull.html), crear las listas de filas y columnas que tienen valores nulos en el conjunto de datos anterior.

Ahora que ya tienes un buen dominio recorriendo la tabla marcando e identificando los valores nulos, es un buen momento para saber que hay una función en Pandas para eliminar los valores marcados como nulos directamente. Es interesante conocer lo anterior porque muchas veces uno se encuentra con datos donde los valores nulos están marcados con otras cosas como la cadena 'NULL' o un -1. En ese caso sería fácil modificar el código anterior para detectarlos haciendo un código 'a medida'. 

Nuestra recomendación en esos casos es que se haga una transformación de esos valores al identificador estándar de Pandas (`np.nan` en características numéricas, None en objetos, NaT en fechas) junto con los procesos de estructurar los datos que cometabamos al final del tema 3. De esta forma, cuando vemos como tratar con los valores nulos, ya tenemos eso bien ordenado y nos podemos centrar en estudiar la mejor estrategia para ellos (eliminarlos o sustituirlos).

La método de los `DataFrame` de Pandas para elimnar valores considerados nulos es: [`dropna`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html#pandas.DataFrame.dropna).

**Ejercicio**: Leer la documentación de [`dropna`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html#pandas.DataFrame.dropna) y averiguar lo que hace el siguiente ejemplo:

In [8]:
tabla_sleep_sin_nulos = tabla_sleep.dropna(axis=0, thresh=8)
tabla_sleep_sin_nulos

,body_weight,brain_weight,slow_wave,paradoxical,total_sleep,maximum_life_span,gestation_time,predation_index,sleep_exposure_index,overall_danger_index
0,6654.000,5712.00,NaN,NaN,3.3,38.6,645.0,3.0,5.0,3.0
1,1.000,6.60,6.3,2.0,8.3,4.5,42.0,3.0,1.0,3.0
2,3.385,44.50,NaN,NaN,12.5,14.0,60.0,1.0,1.0,1.0
4,2547.000,4603.00,2.1,1.8,3.9,69.0,624.0,3.0,5.0,4.0
5,10.550,179.50,9.1,0.7,9.8,27.0,180.0,4.0,4.0,4.0
6,0.023,0.30,15.8,3.9,19.7,19.0,35.0,1.0,1.0,1.0
7,160.000,169.00,5.2,1.0,6.2,30.4,392.0,4.0,5.0,4.0
8,3.300,25.60,10.9,3.6,14.5,28.0,63.0,1.0,2.0,1.0
9,52.160,440.00,8.3,1.4,9.7,50.0,230.0,1.0,1.0,1.0
10,0.425,6.40,11.0,1.5,12.5,7.0,112.0,5.0,4.0,4.0


**Ejercicio**: aplicar `dropna` para eliminar todas las filas con valores nulos.

**Ejercicio**: aplicar `dropna` para eliminar todas las columnas con valores nulos.